# Isogeometric Analysis: Theory and Implementation for the 1D Bar 
## What happens under the hood?
*Mathematical Formulation of IGA, Derivation and Assembly of Stiffness Matrix while solving a simple 1D Bar Example*

### Governing Equation - Strong vs Weak Formulation
The governing differential equation for the static structural problem of the 1D Bar is given by:

$$
Δu(x) = -f(x)
$$

where $Δ$ represents the Laplacian operator. This equation is well known as the Poisson's equation. 

By expanding the laplacian for the 1D Bar problem we end up with the following:

$$
\frac{\partial ^ 2u(x)}{\partial x^2} = -f(x)
$$

This is known as the "Strong form" of the Poisson's equation. This equation can provide the exact solution for particular (finite) points inside the domain or for particular points at the boundaries, caclulated by solving the respective equations for the boundaries (they can be Dirichlet, Neumann or Robin). However, both the domain and boundaries are described by an infinite number of points and in between the selected/particular points for the exact solution of the strong form equations we will end up probably with oscillations, unveiling a huge drawback of the strong form methods. To provide a control between the selected points of the equations the Weak Formulation is recommended instead. As an example we could apply an averaging technique between the particular points of the domain, which is somewhat the case in Finite Element Analysis (FEA) as well. We will not cover the whole derivation procedure for the Weak Formulation of the Poisson's equation but the basic idea behind it is that we multiply the strong form by a test function (the so-called weight function) and integrate over the domain. The test functions are chosen specifically for each problem.


### General Weak Formulation with Dirichlet Boundary Conditions
So, returning back to our methodology, to derive the weak formulation we multiply the strong form by a test function $v(x)$ and integrate over the domain:

$$
\int_Ω  \frac{\partial ^ 2u(x)}{\partial x^2}v(x) dΩ = - \int_Ω f(x)v(x) dΩ
$$

By applying the integration by parts (Green's Theorem) to the left-hand side to move the derivative from $u(x)$ to the test function $v(x)$ we get:

$$
\int_{a}^{b} \frac{\partial^2 u(x)}{\partial x^2}v(x) dx = \left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{a}^{b} - \int_{a}^{b} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} dx 
$$

Here the term $\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{a}^{b}$ is the boundary term evaluated at the boundaries of the domain $a$ and $b$. For the domain $Ω = [0,1]$ this becomes:

$$
\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{0}^{1} - \int_{0}^{1} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} \, dx 
$$

Assuming Dirichlet boundary conditions, where $u(0) = u(1) = 0$ we also consider $v(0)=v(1)=0$. This simplifies the boundary term to zero because the test function $v(x)$ is zero at the boundaries:

$$
\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{0}^{1} = 0
$$

Thus, the weak form of the equation reduces to:
$$
\int_{0}^{1} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} \, dx = \int_{0}^{1} f(x)v(x)\, dx
$$

### Handling Boundary Conditions - Further Explanation (Optional)
In the weak form, the boundary conditions are naturally incorporated through the choice of test functions. For Dirichlet boundary conditions, the test functions are chosen such that they are zero at the boundaries. This ensures that the boundary terms vanish, simplifying the formulation. For Neumann or Robin boundary conditions, the boundary term would not vanish and would need to be included in the weak form. The weak form provides a flexible and stable way to incorporate these conditions and solve the problem numerically.

> This can also be found as:
>$$
\int_Ω \nabla u(x) \nabla v(x) \, dΩ - \int_Ω \frac{\partial u}{\partial n} \, dΓ = \int_Ω f(x)v(x) \, dΩ
>$$
>Given that the boundary condition $v(x) = 0$ on ${\partial Ω}$:
>$$
\int_Ω \nabla u(x) \nabla v(x) \, dΩ = \int_Ω f(x)v(x) \, dΩ
>$$

In the context of the finite element method and isogeometric analysis, the choice of approximating the solution $u(x)$ and the test function $v(x)$ using a finite set of basis function $N_i(x)$ and $N_j(x)$, respectively, is grounded in the method of weighted residuals and the principle of Galerkin's method.

### Weighted Residuals and Galerkin's Method
The primary goal in numerical methods for partial differential equations (PDEs) is to approximate the continuous solution $u(x)$ by a discrete function that satisfies the PDE in an average sense over the domain. This is achieved through the method of weighted residuals, where the residual (the error in satisfying the PDE) is minimized in an integral sense.

#### Weighted Residuals
The idea behind the weighted residuals method is to convert the differential equation into an integral form where the residuals (errors) are weighted and forced to be zero over the domain. The residual $R(u)$ of the differential equation is given by:

$$
R(u) = - \frac{\partial^2u(x)}{\partial x^2} - f(x)
$$

To ensure that the residuals are minimized, we integrate the residual multiplied by a set of weighting functions $v(x)$ over the domain:
$$
\int_Ω R(u)v(x)\, dΩ = 0
$$

#### Galerkin's Method

Bubnov-Galerkin's method is a specific case of the weighted residuals method where the weighting functions are chosen to be the same as the basis functions used to approximate the solution. This means we use the basis functions $N_j(x)$ as our weight functions:
$$
\int_Ω (- \frac{\partial^2u(x)}{\partial x^2} - f(x))N_j(x)\, dΩ = 0
$$

We saw previously the technique for eliminating the second derivative by intergating by parts, thus by applying the same principle for the enclosed in parentheses term we end up with:

$$
\int_Ω \frac{\partial^2u(x)}{\partial x^2} N_j(x) dΩ = -\int_Ω \frac{\partial u(x)}{\partial x} \frac{\partial Ν_j(x)}{\partial x} dΩ
$$

Substituting this into the weighted residuals equation we get:
$$
\int_Ω \nabla u(x) \nabla N_j(x)\, dΩ = \int_Ω f(x)N_j(x)\, dΩ
$$

### Approximation of $u(x)$, $v(x)$ using Basis Functions

Now, we approximate the solution $u(x)$ as a linear combination of basis functions $N_{i}(x)$
$$
u(x) = \sum_{i=Ι} u_iNi(x) 
$$
and
$$
v(x) = N_{j}(x)
$$
$u_{i}$: in case of Isogeometric Analysis these coefficients correspond to the control points. These control points are the unknowns that we solve for in the system of equations. In IGA, the geometry of the domain and the approximation of the solution are both represented using the same set of basis functions (e.g., B-splines or NURBS). The process involves the following steps:

- Geometry Representation: The physical geometry of the domain is defined using control points and basis functions. For example, the shape of the domain is represented as a combination of basis functions and their corresponding control points in the geometric space.
- Solution Approximation: The approximate solution $u(x)$ is expressed as a linear combination of the same basis functions used for geometry, but with a different set of control points that represent the solution values.

Thus, the control points in the solution approximation are analogous to nodal values in the traditional finite element method (FEM). They represent the values of the solution at specific locations in the parametric domain, which are determined by solving the system of equations derived from the weak form of the governing differential equations.

Here’s how it all ties together:

The basis functions $N_{i}(x)$ are predefined and depend on the choice of the knot vector and the degree of the basis functions.
The control points $u_i$ are the unknowns that we need to determine. They scale the basis functions to construct the approximate solution
$u(x)$.

### Formulating the System of Equations
Substituting the approximation into the weak form, we get:
$$
\int_Ω  \nabla (\sum_{i=I} u_iN_i(x)) \nabla N_j(x) dΩ = \int_Ω f(x)N_j(x)dΩ
$$

By linearity of the integral and the dot product, this simplifies to:
$$
\sum_{i=I}ui \int_Ω \nabla (N_i(x)) \nabla N_j(x) dΩ = \int_Ω f(x)N_j(x)dΩ
$$

This leads to a system of linear equations for the coefficients ui:
$$
\sum_{i=I}Kijui = Fj
$$
where
$$
Kij = \int_Ω \nabla Ni(x) \nabla Nj(x) dΩ
$$
$$
Fj = \int_Ω f(x)Nj(x)dΩ
$$

Thus, $Ku = F$.



So now we've done the whole derivation of the Stiffness Matrix but let's continue with a practical example to see how the local stiffness matrix is constructed and how the assembly of the global stiffness matrix is performed.

## Problem Description

Consider a bar of Length L = 1 m, subjected to a uniform load $f(x) = 1$ unit per length. The bar is fixed at both ends with Dirichlet boundary conditions: $u(0) = u(1) = 0$. Solve the above problem using the Isogeometric Analysis with a discretization of 2 elements and B-Spline basis functions.

## Construct the basis functions - Cox 'de Boor Formula
>For degree 0, the basis functions are defined as:
>$$
N_{i,0}(ξ) =
\begin{cases}
1 & \text{if } \xi_i \leq \xi < \xi_{i+1} \\
0 & \text{otherwise}
\end{cases}
>$$
Basis Functions of Degree p:
For p > 0 the basis functions $N_{i,p}(ξ)$ are defined recursively as:
>$$
N_{i,p}(ξ) = \frac{ξ - ξ_i}{ξ_{i+p} - ξ_i}N_{i, p-1}(ξ) + \frac{ξ_{i + p + 1} - ξ}{ξ_{i + p + 1} - ξ_{i + 1}}N_{i + 1, p - 1}(ξ)
>$$

## Calculate the Basis Functions for the 1D Bar Problem


Knot vector:  
$$ξ = [0,0,0,0.5,1,1,1]$$
___



For $ξ_0 = 0$ and $ξ_1 = 0$

$$
N_{0,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0 \\
0 & \text{otherwise}
\end{cases}
$$

So: $N_{0,0}(ξ) = 0$


For $ξ_1 = 0$ and $ξ_2 = 0$

$$
N_{1,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{1,0}(ξ) = 0$


For $ξ_2 = 0$ and $ξ_3 = 0.5$

$$
N_{2,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$


For $ξ_3 = 0.5$ and $ξ_4 = 1$

$$
N_{3,0}(ξ) = 
\begin{cases}
1 & \text{if } 0.5 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$


For $ξ_4 = 1$ and $ξ_5 = 1$

$$
N_{4,0}(ξ) = 
\begin{cases}
1 & \text{if } 1 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{4,0}(ξ) = 0$


For $ξ_5 = 1$ and $ξ_6 = 1$

$$
N_{5,0}(ξ) = 
\begin{cases}
1 & \text{if } 1 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{5,0}(ξ) = 0$

___
>For degree 1, the basis functions are defined as:
>$$
N_{i,1}(ξ) = \frac{ξ - ξ_i}{ξ_{i+1} - ξ_{i}} Ν_{i,0}(ξ) + \frac{ξ_{i+2} - ξ}{ξ_{i+2} - ξ_{i+1}}Ν_{i+1, 0}(ξ)
>$$

Calculate $N_{0,1}$ :

$ξ_{0} = 0$, $ξ_{1} = 0$, $ξ_{2} = 0$

$$
Ν_{0,1}(ξ) = \frac{ξ-ξ_0}{ξ_1-ξ_0}Ν_{0,0}(ξ) + \frac{ξ_2-ξ}{ξ_2-ξ_1}Ν_{1,0}(ξ)
$$

$$
Ν_{0,1}(ξ) = \frac{ξ-0}{0-0}0 + \frac{0-ξ}{0-0}0
$$

Since both terms involve division by zero, we define generally: $N_{0,1}(ξ) = 0$.


Calculate $N_{1,1}$ :

$ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$

$$
Ν_{1,1}(ξ) = \frac{ξ-ξ_1}{ξ_2-ξ_1}Ν_{1,0}(ξ) + \frac{ξ_3-ξ}{ξ_3-ξ_2}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = \frac{ξ-0}{0-0}0 + \frac{0.5-ξ}{0.5-0}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = \frac{0.5-ξ}{0.5}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = (1-2ξ)Ν_{2,0}(ξ)
$$

By substituting $N_{2,0}$ from above:

$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

Calculate $N_{2,1}$ :

$ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$

$$
Ν_{2,1}(ξ) = \frac{ξ-ξ_2}{ξ_3-ξ_2}Ν_{2,0}(ξ) + \frac{ξ_4-ξ}{ξ_4-ξ_3}Ν_{3,0}(ξ)
$$

$$
Ν_{2,1}(ξ) = \frac{ξ-0}{0.5-0}N_{2,0}(ξ) + \frac{1-ξ}{1-0.5}Ν_{3,0}(ξ)
$$

$$
Ν_{2,1}(ξ) = 2ξ Ν_{2,0}(ξ) +(2-2ξ)Ν_{3,0}(ξ) 
$$

By substituting $N_{2,0}$ and $N_{3,0}$ from above:

$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

Calculate $N_{3,1}$ :

$ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$

$$
Ν_{3,1}(ξ) = \frac{ξ-ξ_3}{ξ_4-ξ_3}Ν_{3,0}(ξ) + \frac{ξ_5-ξ}{ξ_5-ξ_4}Ν_{4,0}(ξ)
$$

$$
Ν_{3,1}(ξ) = \frac{ξ-0.5}{1-0.5}N_{3,0}(ξ) + \frac{1-ξ}{1-1}Ν_{4,0}(ξ)
$$

$$
Ν_{3,1}(ξ) = (2ξ-1)Ν_{3,0}(ξ) 
$$

By substituting $N_{3,0}$ from above:

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$


Calculate $N_{4,1}$ :

$ξ_{4} = 1$, $ξ_{5} = 1$, $ξ_{6} = 1$

$$
Ν_{4,1}(ξ) = \frac{ξ-ξ_4}{ξ_5-ξ_4}Ν_{4,0}(ξ) + \frac{ξ_6-ξ}{ξ_6-ξ_5}Ν_{5,0}(ξ)
$$

$$
Ν_{4,1}(ξ) = \frac{ξ-1}{1-1}N_{4,0}(ξ) + \frac{1-ξ}{1-1}Ν_{5,0}(ξ)
$$

$$
Ν_{4,1}(ξ) = 0 
$$

___
>For degree 2, the basis functions are defined as:
>$$
N_{i,2}(ξ) = \frac{ξ - ξ_i}{ξ_{i+2} - ξ_{i}} Ν_{i,1}(ξ) + \frac{ξ_{i+3} - ξ}{ξ_{i+3} - ξ_{i+1}}Ν_{i+1, 1}(ξ)
>$$

Calculate $N_{0,2}$ :

$ξ_{0} = 0$, $ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$

$$
Ν_{0,2}(ξ) = \frac{ξ-ξ_0}{ξ_2-ξ_0}Ν_{0,1}(ξ) + \frac{ξ_3-ξ}{ξ_3-ξ_1}Ν_{1,1}(ξ)
$$

$$
Ν_{0,2}(ξ) = (1 - 2ξ)Ν_{1,1}(ξ)
$$

Given the basis functions N_{0,1}(ξ) = 0 and Ν_{1,1}(ξ):
$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

$$
N_{0,2}(ξ) = 
\begin{cases}
(1 - 2ξ)^2  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

Calculate $N_{1,2}$ :

$ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$

$$
Ν_{1,2}(ξ) = \frac{ξ-ξ_1}{ξ_3-ξ_1}Ν_{1,1}(ξ) + \frac{ξ_4-ξ}{ξ_4-ξ_2}Ν_{2,1}(ξ)
$$

$$
Ν_{1,2}(ξ) = \frac{ξ-0}{0.5-0}Ν_{1,1}(ξ) + \frac{1-ξ}{1-0}Ν_{2,1}(ξ)
$$

$$
Ν_{1,2}(ξ) = 2ξΝ_{1,1}(ξ) + (1-ξ)Ν_{2,1}(ξ)
$$

From the above calculation we know that:
$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
2ξ(1-2ξ) + (1-ξ)2ξ  & \text{if } 0 \leq \xi < 0.5 \\
(1-ξ)(2-2ξ)         & \text{if } 0.5 \leq \xi < 1 \\    
0                   & \text{otherwise}
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
4ξ - 6ξ^2       & \text{if } 0 \leq \xi < 0.5 \\
2 - 4ξ + 2ξ^2   & \text{if } 0.5 \leq \xi < 1 \\    
0               & \text{otherwise}
\end{cases}
$$

Calculate $N_{2,2}$ :

$ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$

$$
Ν_{2,2}(ξ) = \frac{ξ-ξ_2}{ξ_4-ξ_2}Ν_{2,1}(ξ) + \frac{ξ_5-ξ}{ξ_5-ξ_3}Ν_{3,1}(ξ)
$$

$$
Ν_{2,2}(ξ) = \frac{ξ-0}{1-0}Ν_{2,1}(ξ) + \frac{1-ξ}{1-0.5}Ν_{3,1}(ξ)
$$

$$
Ν_{2,2}(ξ) = ξΝ_{2,1}(ξ) + 2(1-ξ)Ν_{3,1}(ξ)
$$

Given that: 
$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
ξ 2ξ                      & \text{if } 0 \leq \xi < 0.5 \\
ξ(2-2ξ) + 2(1-ξ)(2ξ-1)    & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
2ξ^2             & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^2 + 8ξ -2    & \text{if } 0.5 \leq \xi < 1 \\
0                & otherwise
\end{cases}
$$

Calculate $N_{3,2}$ :

$ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$, $ξ_{6} = 1$

$$
Ν_{3,2}(ξ) = \frac{ξ-ξ_3}{ξ_5-ξ_3}Ν_{3,1}(ξ) + \frac{ξ_6-ξ}{ξ_6-ξ_4}Ν_{4,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = \frac{ξ - 0.5}{1 - 0.5}Ν_{3,1}(ξ) + \frac{1 - ξ}{1 - 1}Ν_{4,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = \frac{ξ - 0.5}{0.5}Ν_{3,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = (2ξ - 1)Ν_{3,1}(ξ)
$$

Given that:

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{3,2}(ξ) = 
\begin{cases}
(2ξ-1)^2    & \text{if } 0.5 \leq \xi < 1 \\
0           & otherwise
\end{cases}
$$

___
Summarize the p = 2 Basis Functions:

$$
N_{0,2}(ξ) = 
\begin{cases}
(1 - 2ξ)^2  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
4ξ - 6ξ^2       & \text{if } 0 \leq \xi < 0.5 \\
2 - 4ξ + 2ξ^2   & \text{if } 0.5 \leq \xi < 1 \\    
0               & \text{otherwise}
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
2ξ^2             & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^2 + 8ξ -2    & \text{if } 0.5 \leq \xi < 1 \\
0                & otherwise
\end{cases}
$$

$$
N_{3,2}(ξ) = 
\begin{cases}
(2ξ-1)^2    & \text{if } 0.5 \leq \xi < 1 \\
0           & otherwise
\end{cases}
$$

>*The functions are validated by Journal of Theoretical and Applied Mechanics, Sofia, Vol. 46 No. 4 (2016) pp. 36-75, DOI:10.1515/jtam-2016-0020* 


Calculate the derivatives of the Basis Functions:
$$
\frac {d}{dx} N_{0,2}(ξ) = 
\begin{cases}
8ξ - 4  & \text{if } 0 \leq \xi < 0.5 \\
0       & \text{otherwise}
\end{cases}
$$

$$
\frac {d}{dx} N_{1,2}(ξ) = 
\begin{cases}
4 - 12ξ   & \text{if } 0 \leq \xi < 0.5 \\
-4 + 4ξ   & \text{if } 0.5 \leq \xi < 1 \\    
0         & \text{otherwise}
\end{cases}
$$

$$
\frac {d}{dx} N_{2,2}(ξ) = 
\begin{cases}
4ξ            & \text{if } 0 \leq \xi < 0.5 \\
-12ξ + 8    & \text{if } 0.5 \leq \xi < 1 \\
0             & otherwise
\end{cases}
$$

$$
\frac {d}{dx} N_{3,2}(ξ) = 
\begin{cases}
8ξ - 4    & \text{if } 0.5 \leq \xi < 1 \\
0         & otherwise
\end{cases}
$$




## Calculate the Element's Stiffness Matrix

### Stiffness Matrix in Physical Space
As we saw before, the stiffness matrix $K$ in the physical space is given by:
$$
K_{ij} = \int_Ω \nabla N_{i}(x) \nabla N_{j}(x)\, dΩ
$$

### Transformation to the Parametric Space:
The transformation is done via the Jacobian determinant $J$, which accounts for the change of variables from the parametric space $ξ$ to the physical space $x$. The Jacobian matrix relates the derivatives in the parametric space to the physical space and the typical Jacobian from $(x, y)$ to $(ξ_1, ξ_2)$ is given by:
$$
\begin{equation}
J =
\begin{bmatrix}
\frac{\partial x}{\partial \xi_1} & \frac{\partial x}{\partial \xi_2} \\
\frac{\partial y}{\partial \xi_1} & \frac{\partial y}{\partial \xi_2}
\end{bmatrix}
\end{equation}
$$
The determinant of the Jacobian $det(J(ξ))$ gives the area or volume scale factor.
##### Jacobian Determinant for 1D Bar
##### Mapping Function
For a 1D element, the physical coordinate $x$ is related to the parametric coordinate $ξ$ via a linear mapping inside each element.
- For Element 1:
$$
x = \frac{ξ + 1}{2} \Rightarrow \frac{dx}{dξ} = \frac{1}{2} \Rightarrow J_{element1} = 0.5
$$
- For Element 2:
$$
x = \frac{ξ + 2}{2} \Rightarrow \frac{dx}{dξ} = \frac{1}{2} \Rightarrow J_{element2} = 0.5
$$

### Gradient in Physical Coordinates
The gradient in the physical domain can be expressed in terms of the parametric domain using the Jacobian with the utilization of chain rule:
$$\nabla N_i(x) = \frac{dN_i}{dx} = \frac{dN_i}{dξ}\frac{dξ}{dx} = \frac{dN_i}{dξ}\frac{1}{J}$$
$$
dΩ = J dξ
$$


Substitute the transformed gradients into the stiffness matrix integral:
$$
K_{ij} = \int_{ξ_{min}}^{ξ_{max}} (\frac{dN_{i}}{dξ}\frac{1}{J})(\frac{dN_{j}}{dξ}\frac{1}{J})J\, dξ
$$
$$
K_{ij} = \int_{ξ_{min}}^{ξ_{max}} (\frac{dN_{i}}{dξ})(\frac{dN_{j}}{dξ})\frac{1}{J}\, dξ
$$
$$
K_{ij} = \frac{1}{2}\int_{ξ_{min}}^{ξ_{max}} (\frac{dN_{i}}{dξ})(\frac{dN_{j}}{dξ})\, dξ
$$


---
---
---


### Gaussian Quadrature to calculate the Integral
But what is the well-established porcedure for calculating the integral of the stiffness matrix $K$?
___
*Gaussian quadrature* is a numerical integration technique used to approximate the integral of a function. It is particularly useful for integrals that are difficult to evaluate analytically. The method uses a set of weighted sample points (or nodes) to approximate the integral. Let's discuss this method in here for a quick recap.

For a given quadrature rule, we have a set of quadrature points $ξ_{Κ}$ and weights $W_{K}$:
$$
\int_Ω f(ξ) dΩ \approx \sum_{K} f(ξ_{Κ})W_{K}
$$
So the integral is approximated by the sum of products of the function evaluated at the Gauss points times the respective weight.

For the 1D integration, Gaussian quadrature with 2 points is defined as:
$$
\int_{-1}^{1} f(x)dx = w_1f(x_1) + w_2f(x_2)
$$

This equation is exact for $f(x) = $ $1$, $x$, $x^2$ and $x^3$. Let's examine each case:

$$f(x) = 1 => 2 = w_1 + w_2$$
$$f(x) = x => 0 = w_1x_1 + w_2x_2$$
$$f(x) = x^2 => 2/3 = w_1x_1^2 + w_2x_2^2$$
$$f(x) = x^3 => 0 = w_1x_1^3 + w_2x_2^3$$

$$x_1 = - x_2$$
$$w_1 = w_2$$

$$w_1 = w_2 = 1$$
$$x1 = -1/\sqrt3$$ 
$$x2 = 1/\sqrt3$$

So these are the Gaussian Quadrature Points in the interval [-1, 1] which is the usual case. But do we actually have those points in our domain? The answer is no, so we have to apply a transformation to calculate the Gaussian Quadrature points in the correct interval $\in [0, 1]$.

#### Transform Points from Physical Interval $[a, b]$ to the reference interval $[0, 1]$
We will use a linear mapping. For Gaussian quadrature, we typically have Gauss points defined in the reference interval $[-1, 1]$. However, for the purposes of isogeometric analysis and to match the interval $[0, 1]$, we will transform points accordingly.

Given a point $ξ$ in the interval $[a, b]$, the transformation to the reference interval $[0, 1]$ is:
$$
ξ_{ref} = \frac{ξ - α}{b - a}
$$

#### Transformation from $[-1, 1]$ to $[0, 1]$
For Gaussian quadrature points $ξ_{gauss}$ in the interval $[-1, 1]$, the transformation to $[0, 1]$ is:
$$
ξ_{ref} = \frac{{ξ_{gauss}} + 1}{2}
$$

So using a 2-point Gauss quadrature for the interval $[0, 1]$:
- Gauss points: $$ξ_{1} = \frac{1}{2}(1 - \frac{1}{\sqrt3})$$, $$ξ_{2} = \frac{1}{2}(1 + \frac{1}{\sqrt3})$$
- Gauss weights: $w1 = w = 1$
- NOTE: This is not the case in the 1D Bar example. Here we assume that we have 1 element only for step-by-step demonstration.
  
But we actually have 2 elements: element 1 with $x \in [0, 0.5]$ and element 2 with $x \in [0.5, 1]$ so we are going to use the Gauss points for each element in the local parametric space:

### Reference Gauss Points
As we saw:

$$
ξ_{1, ref} = -\frac{1}{\sqrt3} = -0.577
$$
$$
ξ_{2, ref} = \frac{1}{\sqrt3} = 0.577
$$

The weights $w_1$ and $w_2$ are both 1 for this quadrature.

Element 1 spans [0, 0.5].
Element 2 spans [0.5, 1].

Transformation Formula:
$$
ξ_{Κ} = \frac{(b_{element1} - a_{element1})}{2}j_{k, ref} + \frac{(b_{element2} + a_{element2})}{2}
$$

where $a$ and $b$ are the start and end of the element in physical space.

- Element 1:
    $$
    ξ_{K} = \frac{0.5 - 0}{2}ξ_{Κ, ref} + \frac{0 + 0}{2} = 0.25ξ_{Κ, ref}
    $$
    $$
    ξ_{1, elem1} = 0.106
    $$
    $$
    ξ_{2, elem1} = 0.394
    $$


- Element 2:
    $$
    ξ_{K} = \frac{1 - 0.5}{2}ξ_{Κ, ref} + \frac{1 + 0.5}{2} = 0.25ξ_{Κ, ref} + 0.75
    $$
    $$
    ξ_{1, elem2} = 0.606
    $$
    $$
    ξ_{2, elem2} = 0.894
    $$

Transform the weights also:

$$
w_{k} = \frac{b-a}{2}w_{ref} = 0.25
$$

___


So finally, with the utilization of Gaussian Quadrature the transformed stiffness matrix to the parametric domain is now given by the following sum:
$$
K_{ij} = \frac{1}{J} \sum_{K = 1}^{2}w_{K}(\frac{dN_i}{dξ}|_{ξ_{Κ}}\frac{dN_j}{dξ}|_{ξ_{Κ}})
$$
or by substituting the Jacobian:
$$
K_{ij} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_i}{dξ}|_{ξ_{Κ}}\frac{dN_j}{dξ}|_{ξ_{Κ}})
$$
$$
K_{ij} = 0.125\sum_{K = 1}^{2}(\frac{dN_i}{dξ}|_{ξ_{Κ}}\frac{dN_j}{dξ}|_{ξ_{Κ}})
$$
where $K$ is the transformed Gauss Point into our domain of interest $[0, 1]$.

### 1st Element
For $ξ_{1} = 0.106$ and $ξ_{2} = 0.394$:
$$
\frac{dN_{0,2}}{dξ}|_{ξ1} = 8ξ - 4 = 8 \times 0.106 - 4 = -3.152
$$
$$
\frac{dN_{1,2}}{dξ}|_{ξ1} = 4 - 12ξ = 4 - 12 \times 0.106 = 2.728
$$
$$
\frac{dN_{2,2}}{dξ}|_{ξ1} = 4ξ = 4 \times 0.106 = 0.4226
$$
$$
\frac{dN_{0,2}}{dξ}|_{ξ2} = 8ξ - 4 = 8 \times 0.394 - 4 = -0.848
$$
$$
\frac{dN_{1,2}}{dξ}|_{ξ2} = 4 - 12ξ = 4 - 12 \times 0.394 = -0.728
$$
$$
\frac{dN_{2,2}}{dξ}|_{ξ2} = 4ξ = 4 \times 0.394 = 1.576
$$

Calculate for each pair of basis functions $i, j$:
1. For $K_{00}$:
$$
K_{00} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{00} = 0.125((-3.152)^2 + (-0.848)^2) = 0.125(9.935 + 0.719) = 1.33 
$$

2. For $K_{01}$:
$$
K_{01} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{01} = 0.125((−3.152 \times 2.728) + (-0.848 \times −0.728)) = 0.125(-8.599 + 0.617) = 0.125(-7.982) \approx -1
$$

3. For $K_{02}$:
$$
K_{02} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{02} = 0.125((−3.152 \times 0.424) + (-0.848 \times 1.576)) = 0.125(-1.336 + 1.336) = 0
$$

4. For $K_{11}$:
$$
K_{11} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})^2
$$
$$
K_{11} = 0.125 \times ((2.728)^2 + \times (-0.728)^2) = 0.125 (7.442 + 0.529) = \approx 1
$$

5. For $K_{12}$:
$$
K_{12} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{12} = 0.125 ((2.728 \times 0.42265) + (-0.728 \times 1.576)) = 0.125(1.153 - 1.147) x 0.5 \approx 0  
$$

6. For $K_{22}$:
$$
K_{22} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})^2
$$
$$
K_{22} = 0.125 \times ((0.42265)^2 + (1.57735)^2) = 0.125(0.1786 + 2.488) = 0.333
$$

So finally we end up with the stiffness matrix for the 1st element as:

$$
K_{e1} = 
\begin{bmatrix}
1.333       & -1        & -0.333 \\
-1          & 1         & 0 \\
-0.333      & 0         & 0.333 \\
\end{bmatrix}
$$

For $ξ_{1} = 0.606$ and $ξ_{2} = 0.894$:
$$
\frac{dN_{0,2}}{dξ}|_{ξ1} = 4ξ - 4 = 4 \times 0.606 - 4 = -1.576
$$
$$
\frac{dN_{1,2}}{dξ}|_{ξ1} = 8 - 12ξ = 8 - 12 \times 0.606 = 8 - 7.272 = 0.728
$$
$$
\frac{dN_{2,2}}{dξ}|_{ξ1} = 8ξ - 4 = 8 \times 0.606 - 4 = 0.848
$$
$$
\frac{dN_{0,2}}{dξ}|_{ξ2} = 4ξ - 4 = 4 \times 0.894 - 4 = -0.424
$$
$$
\frac{dN_{1,2}}{dξ}|_{ξ2} = 8 - 12ξ = 8 - 12 \times 0.894 = -2.728 
$$
$$
\frac{dN_{2,2}}{dξ}|_{ξ2} = 8ξ - 4 = 8 \times 0.894 - 4 = 3.152 
$$

### 2nd Element
Calculate for each pair of basis functions $i, j$:
1. For $K_{00}$: 
$$
K_{00} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})^2
$$
$$
K_{00} = 0.125 ((-1.576)^2 + (-0.424)^2) = 0.125(2.483 + 0.179) \approx 0.333
$$

2. For $K_{01}$:
$$
K_{01} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{01} = 0.125((-1.576 \times 0.728) + (-0.424 \times −2.728)) = 0.125(-1.147 + 1.156) \approx 0
$$

3. For $K_{02}$:
$$
K_{02} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{0,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{02} = 0.125((-1.576 \times 0.848) + (-0.424 \times 3.152)) = 0.125(-1.336 - 1.336) \approx -0.333
$$

4. For $K_{11}$:
$$
K_{11} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})^2
$$
$$
K_{11} = 0.125 ((0.728)^2 + (-2.728)^2) = 0.125 (0.53 + 7.44) \approx 1
$$

5. For $K_{12}$:
$$
K_{12} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{1,2}}{dξ}|_{ξ_{Κ}})(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})
$$
$$
K_{12} = 0.125 ((0.728 \times 0.848) + (-2.728 \times 3.152)) = 0.125 (0.6173 - 8.5986) \approx -1
$$

6. For $K_{22}$:
$$
K_{22} = 0.5\sum_{K = 1}^{2}w_{K}(\frac{dN_{2,2}}{dξ}|_{ξ_{Κ}})^2
$$
$$
K_{22} = 0.125 ((0.848)^2 + (3.152)^2) = 0.125 (0.7191 + 9.9351) \approx 1.333
$$

So finally we end up with the stiffness matrix for the 1st element as:

$$
K_{e2} = 
\begin{bmatrix}
0.333       & 0     & -0.333 \\
0           & 1     & -1 \\
-0.333      & -1    & 1.333 \\
\end{bmatrix}
$$




#### Assemblying the Global Stiffness Matrix
To assemble the global stiffness matrix, we need to combine the stiffness matrices of the individual elements, taking into account the connectivity between the elements.

For the 3 knots, the global stiffness matrix $K$ is $3x3$. 

Element 1 connects Nodes 1 and 2, so its contributions go to the (1,1), (1,2), (2,1), (2,2) in the global stiffness matrix.

$$
K = 
\begin{bmatrix}
K_{00}^{(1)} & K_{01}^{(1)} & K_{02}^{(1)} & 0 \\
K_{10}^{(1)} & K_{11}^{(1)} & K_{12}^{(1)} & 0 \\
K_{20}^{(1)} & K_{21}^{(1)} & K_{22}^{(1)} & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix}
$$

Element 2 connects Nodes 2 and 3, so its contributions go to the (2,1), (2,2), (3,1), (3,2) in the global stiffness matrix.

$$
K = 
\begin{bmatrix}
K_{00}^{(1)}    & K_{01}^{(1)}                 & K_{02}^{(1)}                     & 0 \\
K_{10}^{(1)}    & K_{11}^{(1)} + K_{00}^{(2)}  & K_{12}^{(1)} + K_{01}^{(2)}      & K_{12}^{(1)} + K_{02}^{(2)} \\
K_{20}^{(1)}    & K_{21}^{(1)} + K_{10}^{(2)}  & K_{22}^{(1)} + K_{11}^{(2)}      & K_{12}^{(2)} \\
0               & K_{20}^{(2)}                 & K_{21}^{(2)}                     & K_{22}^{(2)} \\
\end{bmatrix}
$$
_____
### Degree Elevation
#### What will happen if we'll use higher order e.g. Cubic basis functions instead of Quadratic?
Let's assume that we want to use Cubic basis functions instead of the Quadratic ones we used in our previous analysis steps.This can be found elsewhere as p-refinement, which is the method where the degree of the basis functions used for representing the geometry is increased. Firstly, we will need to adjust the knot vector for the higher order basis fucntions. When we perform degree elevation from degree $p$ to degree $p + 1$, the multiplicity of the knots is typically increased to maintain the same continuity across the knots. We basically need to increase the multiplicity of all knots to be $m = p + 1$ so that the continuity requirements are met, as before. The knot vector will now become:
$$
ξ = [0, 0, 0, 0, 0.5, 0.5, 1, 1, 1, 1]
$$
___
As we saw previously for quadratic basis functions in each element:
- Element 1 (interval $[0, 0.5]$): Basis functions: $N_{0,2}$, $N_{1,2}$, $N_{2,2}$
- Element 2 (interval $[0.5, 1]$): Basis functions: $N_{1,2}$, $N_{2,2}$, $N_{3,2}$  

The general rule is that each element will have a total of $p + 1$ basis functions where p is the order of basis functions.

Let's now investigate the case for the cubic basis functions. Now each element will have $p + 1 = 3 + 1 = 4$ basis functions. There will be:
- Element 1 (interval $[0, 0.5]$): Basis functions $N_{0,3}$, $N_{1,3}$, $N_{2,3}$, $N_{3,3}$
- Element 2 (interval $[0.5, 1]$): Basis functions $N_{1,3}$, $N_{2,3}$, $N_{3,3}$, $N_{4,3}$

FIXME:
The indexing assumes that we eliminate the zero terms, so it's only accounting for the non-zero contributors to the basis functions. This is easily proven by the fact that when we increased the multiplicity, the basis functions for $p = 0,1,2$ will be the exact same but with their indexes shifted by one position right.

FIXME:
Will I have to calculate all the basis funcs for the knew knot vector -> ASKED.

#### Calculate the Basis Functions of oder $p = 3$
So as before, but now for the non-zero terms we 'll have:
$$
N_{0,2}(ξ) = 
\begin{cases}
(1 - 2ξ)^2      & \text{if } 0 \leq \xi < 0.5 \\
0               & \text{otherwise}
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
4ξ - 6ξ^2       & \text{if } 0 \leq \xi < 0.5 \\
2 - 4ξ + 2ξ^2   & \text{if } 0.5 \leq \xi < 1 \\    
0               & \text{otherwise}
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
2ξ^2             & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^2 + 8ξ -2    & \text{if } 0.5 \leq \xi < 1 \\
0                & otherwise
\end{cases}
$$

$$
N_{3,2}(ξ) = 
\begin{cases}
(2ξ-1)^2    & \text{if } 0.5 \leq \xi < 1 \\
0           & otherwise
\end{cases}
$$

$$
N_{i,3}(ξ) = \frac{ξ - ξ_{i}}{ξ_{i+3} - ξ_{i}}Ν_{i,2}(ξ) + \frac{ξ_{i+4} - ξ}{ξ_{i+4} - ξ_{i+1}}N_{i+1, 2}(ξ)
$$

Calculate $N_{0,3}$ :
$ξ_{0} = 0$, $ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0$, $ξ_{4} = 0.5$
$$
N_{0,3}(ξ) = \frac{ξ - ξ_{0}}{ξ_{3} - ξ_{0}}Ν_{0,2}(ξ) + \frac{ξ_{4} - ξ}{ξ_{4} - ξ_{1}}N_{1, 2}(ξ)
$$
$$
N_{0,3}(ξ) = \frac{ξ - 0}{0 - 0}Ν_{0,2}(ξ) + \frac{0.5 - ξ}{0.5 - 0}N_{1, 2}(ξ)
$$
$$
N_{0,3}(ξ) = (1 - 2ξ)N_{1, 2}(ξ)
$$
$$
N_{0,3}(ξ) = 
\begin{cases}
(1 - 2ξ)(4ξ - 6ξ^2)                 & \text{if } 0 \leq \xi < 0.5 \\
(1 - 2ξ)(2 - 4ξ + 2ξ^2)             & \text{if } 0.5 \leq \xi < 1 \\
0                                   & otherwise
\end{cases}     
$$
$$
N_{0,3}(ξ) = 
\begin{cases}
4ξ - 12ξ^2 + 12ξ^3     & \text{if } 0 \leq \xi < 0.5 \\
2 - 8ξ + 10ξ^2 - 4ξ^3  & \text{if } 0.5 \leq \xi < 1 \\
0                      & otherwise\
\end{cases}
$$

Calculate $N_{1,3}$ :
$ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0$, $ξ_{4} = 0.5$, $ξ_{5} = 1$

$$
N_{1,3}(ξ) = \frac{ξ - ξ_{1}}{ξ_{4} - ξ_{1}}Ν_{1,2}(ξ) + \frac{ξ_{5} - ξ}{ξ_{5} - ξ_{2}}N_{2,2}(ξ)
$$
$$
N_{1,3}(ξ) = \frac{ξ - 0}{0.5 - 0}Ν_{1, 2}(ξ) + \frac{1 - ξ}{1 - 0}N_{2, 2}(ξ)
$$
$$
N_{1,3}(ξ) = 2ξΝ_{1, 2}(ξ) + (1 - ξ)N_{2, 2}(ξ)
$$
$$
N_{1,3}(ξ) = 
\begin{cases}
2ξ(4ξ - 6ξ^2) + (1 - ξ)(2ξ^2)                & \text{if } 0 \leq \xi < 0.5 \\
2ξ(2 -4ξ + 2ξ^2) + (1 - ξ)(-6ξ^2 + 8ξ -2)    & \text{if } 0.5 \leq \xi < 1 \\
0                                            & otherwise\
\end{cases}
$$
$$
N_{1,3}(ξ) = 
\begin{cases}
10ξ^2 - 14ξ^3              & \text{if } 0 \leq \xi < 0.5 \\
14ξ - 22ξ^2 + 10ξ^3 - 2    & \text{if } 0.5 \leq \xi < 1 \\
0                          & otherwise\
\end{cases}
$$

Calculate $N_{2,3}$ :
$ξ_{2} = 0$, $ξ_{3} = 0$, $ξ_{4} = 0.5$, $ξ_{5} = 1$, $ξ_{6} = 1$
$$
N_{2,3}(ξ) = \frac{ξ - ξ_{2}}{ξ_{5} - ξ_{2}}Ν_{2,2}(ξ) + \frac{ξ_{6} - ξ}{ξ_{6} - ξ_{3}}N_{3,2}(ξ)
$$
$$
N_{2,3}(ξ) = \frac{ξ - 0}{1 - 0}Ν_{2, 2}(ξ) + \frac{1 - ξ}{1 - 0}N_{3, 2}(ξ)
$$
$$
N_{2,3}(ξ) = ξΝ_{2, 2}(ξ) + (1 - ξ)N_{3, 2}(ξ)
$$
$$
N_{2,3}(ξ) = 
\begin{cases}
2ξ^3 + (1-ξ)(2ξ - 1)^2      & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^3 + 8ξ^2 - 2ξ           & \text{if } 0.5 \leq \xi < 1 \\
0                           & otherwise\
\end{cases}
$$

$$
N_{2,3}(ξ) = 
\begin{cases}
-2ξ^3 + 8ξ^2 - 5ξ + 1       & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^3 + 8ξ^2 - 2ξ           & \text{if } 0.5 \leq \xi < 1 \\
0                           & otherwise\
\end{cases}
$$

Calculate $N_{3,3}$ :
$ξ_{3} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$, $ξ_{6} = 1$
$$
N_{3,3}(ξ) = \frac{ξ - ξ_{3}}{ξ_{6} - ξ_{3}}Ν_{3,2}(ξ) + \frac{ξ_{7} - ξ}{ξ_{7} - ξ_{4}}N_{4,2}(ξ)
$$
$$
N_{3,3}(ξ) = \frac{ξ - 0.5}{1 - 0.5}Ν_{3, 2}(ξ) + \frac{1 - ξ}{1 - 1}N_{4, 2}(ξ)
$$

Since $ξ6 = ξ7 = 1$, we observe that:
$$
N_{4,2}(ξ) = 0 
$$

This simplifies $N_{3,3}(ξ)$ to:

$$
N_{3,3}(ξ) = (2ξ - 1)Ν_{3, 2}(ξ)
$$

$$
N_{3,3}(ξ) = 
\begin{cases}
0             & \text{if } 0 \leq \xi < 0.5 \\
(2ξ - 1)^3    & \text{if } 0.5 \leq \xi < 1 \\
0             & otherwise\
\end{cases}
$$

And their respective derivatives:
$$
\frac{dN_{0,3}(ξ)}{dξ} = 
\begin{cases}
4 - 24ξ + 36ξ^2   & \text{if } 0 \leq \xi < 0.5 \\
-8 + 20ξ - 12ξ^2  & \text{if } 0.5 \leq \xi < 1 \\
0                 & otherwise\
\end{cases}
$$
$$
\frac{dN_{1,3}(ξ)}{dξ} = 
\begin{cases}
20ξ - 42ξ^2        & \text{if } 0 \leq \xi < 0.5 \\
14 - 44ξ + 30ξ^2   & \text{if } 0.5 \leq \xi < 1 \\
0                  & otherwise
\end{cases}
$$
$$
\frac{dN_{2,3}(ξ)}{dξ} = 
\begin{cases}
-6ξ^2 + 16ξ - 5     & \text{if } 0 \leq \xi < 0.5 \\
-18ξ^2 + 16ξ - 2    & \text{if } 0.5 \leq \xi < 1 \\
0                   & otherwise
\end{cases}
$$
$$
\frac{dN_{3,3}(ξ)}{dξ} = 
\begin{cases}
0               & \text{if } 0 \leq \xi < 0.5 \\
6(2ξ -1)^2      & \text{if } 0.5 \leq \xi < 1 \\
0               & otherwise
\end{cases}
$$

As we see, there are intervals of coverage for $[0, 0.5]$ and $[0.5, 1]$ to all of the above functions from $\frac{dN_{0,3}(ξ)}{dξ}$ to $\frac{dN_{3,3}(ξ)}{dξ}$. So there will be an increase in size in the local stiffness matrix for each element and from $3x3$ in the 2nd order basis functions we end up with a local stiffness matrix of $4x4$. The stiffness matrix for the 3rd order basis functions:

$Κ_{00}$
$Κ_{01}$
$Κ_{02}$
$Κ_{03}$

$Κ_{10}$
$Κ_{11}$
$Κ_{12}$
$Κ_{13}$

$Κ_{20}$
$Κ_{21}$
$Κ_{22}$
$Κ_{23}$

$Κ_{30}$
$Κ_{31}$
$Κ_{32}$
$Κ_{33}$

_____
#### Strain-Displacement Matrix
What is it, why is it introduced and will i use it in 1D ?

The strain-displacement matrix, often deonted as $B$, relates the strains in the element to the nodal displacements. For 1D bar element, the strain $ε$ is given by the derivative of the displacement field $u(x)$ with respect to the spatial coordinate $x$:
$$
ε = \frac{du}{dx}
$$
In Isogeometric Analysis (IGA) and Finite Element Analysis (FEA), the strain-displacement matrix (also known as the strain-displacement relationship) is used to connect the nodal displacements to the strains within an element. It is essential for formulating the system of equations that describes the mechanical behavior of the structure.

Need for the Strain-Displacement Matrix
For Linear Analysis: In linear problems where the material behavior is linear and the displacement fields are linear, the stiffness matrix directly relates nodal forces to displacements. The strain-displacement matrix is not separately required because the stiffness matrix incorporates the necessary strain-displacement relationships.

For Nonlinear Analysis or Complex Behavior: In more complex scenarios, such as nonlinear materials or higher-order analyses, the strain-displacement matrix becomes crucial. It helps to explicitly relate the strain fields to the displacement fields.

In the Context of Your Current Calculation
1D Bar Element: For a 1D bar element with linear shape functions, the stiffness matrix you calculate already incorporates the strain-displacement relationship implicitly. Since the strain in a 1D bar is simply the gradient of the displacement field, and the shape functions are linear, the calculation of the stiffness matrix directly reflects this relationship.

No Need for Explicit Strain-Displacement Matrix: In your case, since you are dealing with linear shape functions and a straightforward 1D bar element, the stiffness matrix derived using Gaussian quadrature already represents the effect of strains. The matrix you computed is already reflecting the strain-displacement relationship.

Summary
In Your Case: There is no need to explicitly write out the strain-displacement matrix because the stiffness matrix calculation inherently includes the strain-displacement relationship for a 1D linear element.
___
For More Complex Elements: If you were dealing with higher-dimensional problems, nonlinear materials, or nonlinear shape functions, then expressing and utilizing the strain-displacement matrix would be more relevant and necessary.

Thus, you can proceed with assembling the global stiffness matrix using the stiffness matrices you computed for each element without needing to separately express the strain-displacement matrix.

Is there a way to think to assemble the stiffness matrix efficiently when the order increases? Is there any parallel approach that should be used? Please explain by words, no code needed at this step.
ChatGPT
As the order of basis functions increases in Finite Element Analysis (FEM) or Isogeometric Analysis (IGA), the complexity of the stiffness matrix computation indeed increases. Here are some strategies and considerations for assembling the stiffness matrix efficiently, especially when dealing with higher-order basis functions:

Strategies for Efficient Assembly
Efficient Integration:

Adaptive Quadrature: Use adaptive quadrature methods to allocate more integration points where the function changes rapidly, which improves accuracy without significantly increasing the number of points overall.
Higher-Order Quadrature: For higher-order basis functions, you may need higher-order quadrature rules. Ensure that the chosen quadrature scheme is sufficient to handle the complexity of the basis functions.
Matrix Sparsity:

Exploiting Sparsity: The stiffness matrix is often sparse, meaning many of its entries are zero. This sparsity can be exploited to save memory and computational time. Utilize sparse matrix data structures and operations to handle large matrices efficiently.
Band Structure: For 1D problems, matrices typically have a banded structure, which can be leveraged to reduce computational complexity.
Assembly Process Optimization:

Local Assembly: Compute the element stiffness matrices locally and then assemble them into the global matrix. This localized approach helps manage memory and computational resources more effectively.
Efficient Indexing: Use efficient indexing schemes to assemble the global matrix from local element matrices, minimizing the overhead involved in matrix operations.
Parallel Processing:

Element-wise Parallelism: Since the computation for each element is independent of others, the assembly process can be parallelized. Assign different elements to different processors or threads.
Integration Parallelism: Numerical integration over each element can also be parallelized, where different integration points or elements are processed simultaneously.
Matrix Compression and Storage:

Sparse Matrix Formats: Use compressed sparse row (CSR) or compressed sparse column (CSC) formats to store and manipulate sparse matrices efficiently.
Efficient Memory Usage: Employ techniques such as block storage or hierarchical data structures to manage large matrices and reduce memory overhead.
Algorithmic Improvements:

Fast Matrix Operations: Use optimized libraries for matrix operations, such as BLAS or LAPACK, which are implemented with high efficiency and can handle large-scale problems effectively.
Preconditioning: For iterative solvers, use preconditioners to improve convergence rates and reduce computational time.
Parallel Approaches
Domain Decomposition:

Spatial Decomposition: Divide the domain into smaller subdomains that can be processed independently. Each subdomain can be handled by a separate processor, with communication between processors to handle boundary conditions and data exchange.
Task Parallelism:

Separate Tasks: Break down the assembly process into separate tasks (e.g., integration, matrix assembly) that can be executed concurrently. Distribute these tasks across multiple processors or cores.
Data Parallelism:

Matrix Operations: For matrix operations that are inherently parallel, such as matrix multiplication or factorization, use parallel algorithms to speed up computations.
GPU Acceleration:

Leverage GPUs: Use Graphics Processing Units (GPUs) for highly parallelizable operations, such as matrix assembly and integration. GPUs can handle large-scale computations more efficiently than CPUs in many cases.
Summary
In summary, when dealing with higher-order basis functions and the corresponding stiffness matrices, the following approaches can enhance efficiency:

Use adaptive or higher-order quadrature for accurate integration.
Exploit matrix sparsity and banded structures.
Optimize the assembly process with local computations and efficient indexing.
Utilize parallel processing for both element computations and integration.
Apply matrix compression techniques and optimized libraries.
Consider domain decomposition and task parallelism for large-scale problems.
By incorporating these strategies, you can handle the increased complexity of higher-order basis functions more efficiently and effectively.

TO_DO:
-2D Plate Example with Isogeometric Analysis - Capture the differences.
-Examine the theory of assemblying the stiffness matrix based on the PhD thesis provided.
OPTIONAL:
- Python live script which will be deployed, to create the B-Splines and control them.
- Python script to visualize each step of the procedure and how the control points will be moved after the solution and how the new geometry will be described.
- Write the sectors which should be parallel. There is no interdependency so it is easy to parallelize.